In [1]:
# Imports necessary modules from various libraries 
from Bio.PDB import PDBParser, MMCIFParser, MMCIFIO, Select 
import numpy as np
import os
from itertools import combinations
from collections import defaultdict

# Stores detected interfacial chains by symmetry 
interfaces = {
    "dimer": [],
    "trimer": [],
    "tetramer": [],
    "pentamer": []
    }

# Declares a dictionary
global interfaces

# Defines label assignments for each interface type
label_map_upper = {
    "dimer": ['A', 'B'],
    "trimer": ['A', 'C', 'D'],
    "tetramer": ['A', 'E', 'F'],
    "pentamer": ['A', 'G', 'H'],
}

# Generates corresponding lowercase labels for overlapping chains
label_map_lower = {
    k: [l.lower() for l in v] for k, v in label_map_upper.items()
}

In [2]:
# Allows relabeling of chains when saving the structure 
class ChainRelabelSelect(Select):
    def __init__(self, relabel_map):
        self.relabel_map = relabel_map # Dictionary mapping old chain IDs to new ones

    def accept_chain(self, chain):
        return True

    def get_chain_id(self, chain):
        return self.relabel_map.get(chain.id, chain.id)

In [3]:
### Auxiliary functions

# Returns a chain object from a structure object given a chain ID string
def get_chain_by_id(structure, chain_id):
    for chain in structure.get_chains():
        if chain.id == chain_id:
            return chain

# Identifies if a structure is a homomer or a heteromer by comparing the first chain against all other chains. Returns "Homomer" or "Heteromer"
def determine_structure_type(structure):
    chain_residues = []
    homomer = True

    # Creates a list of lists of residues for each chain
    for chain in structure.get_chains():
        residues = []
        for residue in chain.get_residues():
            if residue.id[0] == ' ':
                residues.append(residue.get_resname())
        chain_residues.append(residues)

    # Compares the first chain sequence against all other chain sequences
    for chain in chain_residues[1:]:
        if chain_residues[0] != chain:
            homomer = False

    if homomer:
        print("Structure is a homomer.")
        return "Homomer"
    else:
        print("Structure is a heteromer.")
        return "Heteromer"

# Returns centroid coordinates given a chain object
def chain_centroid(chain):
    coords = []
    for atom in chain.get_atoms():
        coords.append(atom.coord)
    centroid = np.array(coords).mean(axis=0)
    return centroid

# Returns a list of chain ID strings that neighbor the input chain object and the centroid coordinates of the input chain object
def find_neighbors(structure, origin_chain):
    neighboring_chains = set()
    centroid = chain_centroid(origin_chain)

    # Adding a chain ID string to a list if one of its atoms are close enough (< 25) to the centroid of the input chain object
    for chain in structure:
        for atom in chain.get_atoms():
            distance = np.linalg.norm(atom.coord - centroid)
            if distance <= 25:
                neighboring_chains.add(atom.get_full_id()[2])

    neighboring_chains = list(neighboring_chains)
    neighboring_chains.remove(origin_chain.id)
    return neighboring_chains, centroid

# Calculates the angle between two vectors
def calculate_angle(vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    magnitude1 = np.linalg.norm(vector1)
    magnitude2 = np.linalg.norm(vector2)
    return np.degrees(np.arccos(dot_product / (magnitude1 * magnitude2)))

# Checks angles for symmetry
def angle_check(angle):
    if 59.75 < angle < 60.25:  # trimer
        return "trimer"
    elif 89.75 < angle < 90.25:  # tetramer
        return "tetramer"
    elif 107.75 < angle < 108.25:  # pentamer
        return "pentamer"

# Analyzes the angles from a list of vectors and returns a list of used chains and lists of trimers, tetramers, and pentamers
def analyze_angles(vectors, neighbors):
    used_chains, trimers, tetramers, pentamers = [], [], [], []
    for i in range(len(vectors) - 1): # Nested for loop to compare all vectors
        for j in range(i, len(vectors)):
            if i != j:
                angle = calculate_angle(vectors[i], vectors[j])
                if angle_check(angle) == "trimer":
                    used_chains.extend([neighbors[i], neighbors[j]])
                    trimers.extend([neighbors[i], neighbors[j]])
                    print(f"{neighbors[i]} and {neighbors[j]} are trimeric.")
                elif angle_check(angle) == "tetramer":
                    used_chains.extend([neighbors[i], neighbors[j]])
                    tetramers.extend([neighbors[i], neighbors[j]])
                    print(f"{neighbors[i]} and {neighbors[j]} are tetrameric.")
                elif angle_check(angle) == "pentamer":
                    used_chains.extend([neighbors[i], neighbors[j]])
                    pentamers.extend([neighbors[i], neighbors[j]])
                    print(f"{neighbors[i]} and {neighbors[j]} are pentameric.")
    return used_chains, trimers, tetramers, pentamers

# Determines if two chains are a homodimer based on the distance of an atom from the midpoint of the two chains and returns True or False
def dimer_check(first_chain, comparison_chain):
    first_atom = next(first_chain.get_atoms()).coord
    comparison_atom = next(comparison_chain.get_atoms()).coord
    midpoint = (chain_centroid(first_chain) + chain_centroid(comparison_chain)) / 2
    dist1 = np.linalg.norm(first_atom - midpoint)
    dist2 = np.linalg.norm(comparison_atom - midpoint)
    if np.linalg.norm(dist1 - dist2) < 0.1:
        return True
    else:
        return False

# Determines the symmetric neighboring chains given an input structure and input chain object. Returns a list of neighbors, dimers, trimers, tetramers, and pentamers. Lists may be empty
def chain_analysis(structure, chain):
    vectors, dimer = [], []
    print(f"\nChain of comparison is {chain.id}.")
    neighbors, centroid = find_neighbors(structure, chain) # Finds neighbors to input chain

    for neighbor in neighbors: # Determines the vectors of all neighboring chains
        vectors.append(chain_centroid(get_chain_by_id(structure, neighbor)) - centroid)

    used_chains, trimer, tetramer, pentamer = analyze_angles(vectors, neighbors)

    for used in used_chains: # Removes any chains designated as trimer, tetramer, or pentamer
        neighbors.remove(used)

    for neighbor in neighbors: # Checks for any dimers
        if dimer_check(chain, get_chain_by_id(structure, neighbor)):
            dimer.append(neighbor)
            print(f"{neighbor} is dimeric.")

    for dimers in dimer: # Removes any chains designated as dimers - necessary for heteromers only
        neighbors.remove(dimers)

    return neighbors, dimer, trimer, tetramer, pentamer

def relabel_structure(input_file):

    # Determines file type and chooses appropriate parser 
    ext = os.path.splitext(input_file)[1].lower()
    if ext == '.cif':
        parser = MMCIFParser(QUIET=True)
    elif ext == '.pdb':
        parser = PDBParser(QUIET=True)
    else:
        raise ValueError("Unsupported file format. Use .pdb or .cif")

    # Parse structure
    structure = parser.get_structure("protein", input_file)

    # Determine if it's a homomer or heteromer
    structure_type = determine_structure_type(structure)

    # Process structure differently based on type
    if structure_type == "Homomer":
        process_homomer(structure)
    else:
        process_heteromer(structure) 

    relabel_map = {} # Mapping from old to new chain IDs
    used_chains = set() # Tracks already relabeled chains
    main_chain_id = next(structure.get_chains()).id # Uses first chain as a reference

    # Assign labels to detected interface chains
    for interface_type, interfaces_of_type in interfaces.items():
        for chain_combo, _ in interfaces_of_type:
            is_secondary = chain_combo[0] != main_chain_id and structure_type == "Heteromer"
            labels = label_map_lower[interface_type] if is_secondary else label_map_upper[interface_type]
            for i, chain_id in enumerate(chain_combo):
                if chain_id not in relabel_map:
                    relabel_map[chain_id] = labels[i % len(labels)]
                    used_chains.add(chain_id)

    # Assign 'z', 'z1', etc to leftover chains
    z_count = 0
    for chain in structure[0]:
        if chain.id not in relabel_map:
            label = 'z' if z_count == 0 else f'z{z_count}'
            relabel_map[chain.id] = label
            z_count += 1

    # Apply relabeling to the structure 
    for model in structure:
        for chain in model:
            if chain.id in relabel_map:
                chain.id = relabel_map[chain.id]

    # Save updated structure to new CIF file
    output_file = os.path.splitext(input_file)[0] + "_relabel.cif"
    io = MMCIFIO()
    io.set_structure(structure)
    io.save(output_file, ChainRelabelSelect(relabel_map))

    # prints the file name to which the output is written
    print(f"✅ Output written to: {output_file}")   

In [4]:
def process_homomer(structure):
    # Analyzes the structure to identify symmetrical interfaces around the first chain
    _, dimers, trimers, tetramers, pentamers = chain_analysis(structure, next(structure.get_chains()))
    
    # Stores dimer interfaces (if found)
    for d in dimers:
        interfaces["dimer"].append(([next(structure.get_chains()).id, d], None))
    
    # Stores trimer interfaces (if found)
    if trimers:
        interfaces["trimer"].append(([next(structure.get_chains()).id] + list(set(trimers)), None))
        
    # Stores tetramer interfaces (if found)
    if tetramers:
        interfaces["tetramer"].append(([next(structure.get_chains()).id] + list(set(tetramers)), None))
        
    # Stores pentamer interfaces (if found)
    if pentamers:
        interfaces["pentamer"].append(([next(structure.get_chains()).id] + list(set(pentamers)), None))

In [5]:
def process_heteromer(structure):
    # Identifies symmetry for first chain type
    neighbors, dimers1, trimers1, tetramers1, pentamers1 = chain_analysis(structure, next(structure.get_chains()))
    residues, chain_residues = [], []

    # Identifies different interfacing chain
    for residue in next(structure.get_chains()):
        residues.append(residue.get_resname())
    chain_residues.append(residues)

    for remaining in neighbors:
        residues = []
        for residue in get_chain_by_id(structure, remaining):
            if residue.id[0] == ' ': # standard residue
                residues.append(residue.get_resname())
        chain_residues.append(residues)

    for index, chain in enumerate(chain_residues):
        if chain_residues[0][0] != chain[0]:
            other_chain = get_chain_by_id(structure, neighbors[index - 1])

    print(f"{other_chain.id} interfaces with {next(structure.get_chains()).id}.")
    neighbors.remove(other_chain.id)

    for d in dimers1:
        interfaces["dimer"].append(([next(structure.get_chains()).id, d], None))
    if trimers1:
        interfaces["trimer"].append(([next(structure.get_chains()).id] + list(set(trimers1)), None))
    if tetramers1:
        interfaces["tetramer"].append(([next(structure.get_chains()).id] + list(set(tetramers1)), None))
    if pentamers1:
        interfaces["pentamer"].append(([next(structure.get_chains()).id] + list(set(pentamers1)), None))

    # Identifies symmetry for second chain type
    neighbors, dimers2, trimers2, tetramers2, pentamers2 = chain_analysis(structure, other_chain)

    for d in dimers2:
        interfaces["dimer"].append(([other_chain.id, d], None))
    if trimers2:
        interfaces["trimer"].append(([other_chain.id] + list(set(trimers2)), None))
    if tetramers2:
        interfaces["tetramer"].append(([other_chain.id] + list(set(tetramers2)), None))
    if pentamers2:
        interfaces["pentamer"].append(([other_chain.id] + list(set(pentamers2)), None))

In [6]:
# Test cases
relabel_structure("5cy5.cif")
relabel_structure("7sge.cif")
relabel_structure("5im5.cif")
relabel_structure("9fo3.cif")
relabel_structure("1dat.cif")
relabel_structure("6_gen_I_125_NCN_2_28.pdb")

Structure is a heteromer.

Chain of comparison is B.
B-12 and B-6 are trimeric.
A interfaces with B.

Chain of comparison is A.
A-7 and A-10 are trimeric.


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/Bio/PDB/Entity.py:197: BiopythonWarning: The id `A` is already used for a sibling of this entity. Changing id from `B` to `A` might create access inconsistencies to children of the parent entity.
  warnings.warn(


✅ Output written to: 5cy5_relabel.cif
Structure is a heteromer.

Chain of comparison is A.
A-23 and A-10 are trimeric.
A-6 is dimeric.
B interfaces with A.

Chain of comparison is B.
B-2 and B-5 are pentameric.


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/Bio/PDB/Entity.py:197: BiopythonWarning: The id `C` is already used for a sibling of this entity. Changing id from `A-23` to `C` might create access inconsistencies to children of the parent entity.
  warnings.warn(


✅ Output written to: 7sge_relabel.cif
Structure is a heteromer.

Chain of comparison is K.
M and S are pentameric.
U interfaces with K.

Chain of comparison is U.
W and V are trimeric.
U-4 is dimeric.


...

✅ Output written to: 9fo3_relabel.cif
Structure is a homomer.

Chain of comparison is A.
A-15 and A-16 are tetrameric.
A-9 and A-5 are trimeric.
A-22 is dimeric.


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/Bio/PDB/Entity.py:197: BiopythonWarning: The id `C` is already used for a sibling of this entity. Changing id from `A-9` to `C` might create access inconsistencies to children of the parent entity.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/Bio/PDB/Entity.py:197: BiopythonWarning: The id `D` is already used for a sibling of this entity. Changing id from `A-10` to `D` might create access inconsistencies to children of the parent entity.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/Bio/PDB/Entity.py:197: BiopythonWarning: The id `B` is already used for a sibling of this entity. Changing id from `A-22` to `B` might create access inconsistencies to children of the parent entity.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/Bio/PDB/Entity.py:197: BiopythonW

✅ Output written to: 1dat_relabel.cif
Structure is a homomer.

Chain of comparison is A.
T and Y are pentameric.
i and k are trimeric.
s is dimeric.


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/Bio/PDB/Entity.py:197: BiopythonWarning: The id `A` is already used for a sibling of this entity. Changing id from `B` to `A` might create access inconsistencies to children of the parent entity.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/Bio/PDB/Entity.py:197: BiopythonWarning: The id `z` is already used for a sibling of this entity. Changing id from `C` to `z` might create access inconsistencies to children of the parent entity.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/Bio/PDB/Entity.py:197: BiopythonWarning: The id `A` is already used for a sibling of this entity. Changing id from `K` to `A` might create access inconsistencies to children of the parent entity.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/Bio/PDB/Entity.py:197: BiopythonWarning: 

✅ Output written to: 6_gen_I_125_NCN_2_28_relabel.cif
